In [21]:
import sys 
sys.path.insert(0,"../")
from src.storage.database import websites_db
from gensim.models.fasttext import FastText
from gensim.models import fasttext
from nltk.corpus import stopwords 
from gensim.test.utils import datapath
from keybert import KeyBERT
from src.index.preprocessing_pipeline import Pipeline
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
ft_model = fasttext.load_facebook_vectors('../data/wiki.en.bin')

In [9]:
ft_model_mine = FastText.load('../data/version_1/ft_model.m')

In [12]:
model = KeyBERT('xlm-r-distilroberta-base-paraphrase-v1')

In [13]:
with open('../republic.txt','r') as f:
    doc = ' '.join(f.readlines())
keywords = model.extract_keywords(doc)
keywords

[('millennia', 0.3567),
 ('centuriesâ', 0.3457),
 ('centuries', 0.3395),
 ('liberty', 0.3362),
 ('millennium', 0.3356)]

In [34]:
bi_words = model.extract_keywords(doc, keyphrase_ngram_range=(1, 2))

In [43]:
bi_words.append(("test query",1.0))
bi_words.append(("palpatine",1.0))

In [44]:
piped_doc = " ".join(list(Pipeline(set(stopwords.words("english"))).pipe(doc)))
doc_enc_mine = [ft_model_mine.wv[vec] for vec in piped_doc.split()]
doc_enc_mine = np.mean(doc_enc_mine,axis=0)

doc_enc = [ft_model[vec] for vec in piped_doc.split()]
doc_enc = np.mean(doc_enc,axis=0)

In [45]:
for words in bi_words:
    score = words[1]
    words = words[0]     
    query = [ft_model[vec] for vec in words.split()]
    query = np.mean(query,axis=0)
    similarity = cosine_similarity(np.array([doc_enc, query]))[0][1]
    print(similarity,score, words)


0.6696127 0.5621 republic galactic
0.6404427 0.5195 republic galaxy
0.6344273 0.4957 republic millennia
0.6319883 0.487 centuries galactic
0.56188 0.4868 civilizations galaxy
0.37439156 1.0 test query
0.37439156 1.0 test query
0.57914734 1.0 palpatine


In [46]:
for words in bi_words:
    score = words[1]
    words = words[0]     
    query = [ft_model_mine.wv[vec] for vec in words.split()]
    query = np.mean(query,axis=0)
    similarity = cosine_similarity(np.array([doc_enc_mine, query]))[0][1]
    print(similarity,score, words)


0.8631183 0.5621 republic galactic
0.7601992 0.5195 republic galaxy
0.8555117 0.4957 republic millennia
0.93063307 0.487 centuries galactic
0.84605175 0.4868 civilizations galaxy
0.79870725 1.0 test query
0.79870725 1.0 test query
0.7370023 1.0 palpatine


In [148]:

input = "galactic republic".lower().split()
input_2 = "the empire".lower().split()
query = [ft_model.wv[vec] for vec in input]
query = np.mean(query,axis=0)
query_2 = [ft_model.wv[vec] for vec in input_2]
query_2 = np.mean(query_2,axis=0)

In [157]:

x = cosine_similarity(np.array([doc_enc, query, query_2]))
# doc <-> query 84.75, query 2 -> 73
# query <-> query 60
print('pairwise sparse output:\n {}\n'.format(x[0]))

pairwise sparse output:
 [1.0000004 0.8631183 0.8338314]



In [138]:
 model.extract_keywords(piped_doc, keyphrase_ngram_range=(4, 5))

[('galactic republic galactic republic time', 0.7296),
 ('republic common galactic republic', 0.7265),
 ('used republic common galactic republic', 0.7224),
 ('galactic republic galactic republic', 0.7211),
 ('republic galactic republic time', 0.7182)]

In [130]:
piped_doc

'galactic republic 25053 bby–19 bby ruling government galaxy existed twentyfive thousand year reformation galactic empire 19 bby throughout existence state commonly known republic postrepublic era beyond remembered old republic also rarely known old galactic republic first galactic republic time term old order used describe republic common galactic republic representative democracy composed various star system sector member world throughout known galaxy central government consisted three branch legislative executive judicial first galactic senate body elected appointed representative main duty included mediating dispute issue ranging planetary galacticscale regulating trade route star system theory represented interest republic citizen office supreme chancellor senate executive counterpart supreme chancellor elected rank senate member leader senate overall representative republic chancellor also held title head state president judicial branch consisted supreme court lower regional cour